# Traveling salesman with GA 

### This notebook covers the creation of a Genetic Algorithm to tackle the TSP problem

Feel free to copy it and sugest improvements on github
https://github.com/jcanelhas/GAIntro

### Module Import

The imports go here

In [ ]:
#numpy 
import numpy as np

#some pretty plots
import matplotlib as plt

#notebook or inline 
%matplotlib inline
plt.rcParams["figure.figsize"]=(16,8)

#general math and random imports
from math import sqrt
from random import randint
from random import random

#For some cool widgets
from ipywidgets import FloatProgress
from IPython.display import display

### Read the cities file

Read a text file into an array

In [ ]:
city_list=np.loadtxt('200.txt').astype(int)
print(str(len(city_list)) + " cities read")
#show the map
for city in city_list:
    plt.pyplot.plot(city[1],city[2],'.')
    plt.pyplot.text(1,6000,'Cities')
plt.pyplot.show()

### Set the initial parameters

In [ ]:
#Number of individuals on a population
nindividuals=300
#Number of cities being processed
ncities=len(city_list)
#Tournmentsize
tournamentsize=50

#crossover and mutation probability
cx_probability=0.9
mx_probability=0.5

#maximum number of generations to run
max_iterations=1000

## Define general funtions

### Euclidean Distance

In [ ]:
def euclidian_distance(x1,y1,x2,y2):
    dist= sqrt((x1-x2)**2+(y1-y2)**2)
    return dist

In [ ]:
#test the function
euclidian_distance(0,0,1,1)

### Calculate fitness

In [ ]:
def calculate_fitness(individual):
    fitness=0
    #First element
    fitness=euclidian_distance(city_list[individual[0]][1],city_list[individual[0]][2],
                               city_list[individual[1]][1],city_list[individual[0]][2])
    for i in range(1,len(individual)-1):
        fitness=fitness+euclidian_distance(city_list[individual[i]][1],city_list[individual[i]][2],
                                           city_list[individual[i+1]][1],city_list[individual[i+1]][2])
    return fitness

### Mutation

In [ ]:
def mutate(individual):
    #reverse mutation
    init_pos=np.random.randint(0,len(individual))
    end_pos=np.random.randint(init_pos,len(individual)+1)
    mutated_individual= np.concatenate((individual[:init_pos],
                                        individual[init_pos:end_pos][::-1],
                                        individual[end_pos:]),axis=0)
    return mutated_individual

### Crossover

In [ ]:
def crossover(individual1,individual2):
    splitpoint=randint(0,len(individual1)-1)
    result1=np.copy(individual1)
    result2=np.copy(individual2)
    currpos=splitpoint
    firstval=individual1[splitpoint]
    newpos=0
    
    cycleover=False
    
    while True:
         
            
        result1[currpos]=individual2[currpos]
        result2[currpos]=individual1[currpos]



        newpos=np.where(individual1==individual2[currpos])

        currval=individual2[currpos]

        if firstval==currval:#individual2[currpos]:

            cycleover=True        

        currpos=newpos
        if cycleover:
            break

   
    return(result1,result2)

### Tournament Selection

Tournament selections randomly picks a number of individuals and compares them, the one with the best fitness is selected. This method is fast compared to roulette wheel (and others )because there is no need to evaluate EVERY individual in the population.

In [ ]:
def tournament_selection(pop, tournament_size):
    
    champion=population[np.random.randint(len(pop))]
    champion_fitness=calculate_fitness(champion)
    
    for i in range(0,tournament_size):
        tmpind=population[np.random.randint(len(pop))]
        tmpind_fitness=calculate_fitness(tmpind)
        if tmpind_fitness<champion_fitness:
            champion=tmpind[:]
            champion_fitness=tmpind_fitness
    return champion

### Initialization

In [ ]:
population=np.zeros((nindividuals,ncities),dtype=int)
for individual in range(0,nindividuals):
    population[individual]=np.arange(ncities)
    np.random.shuffle(population[individual])

In [ ]:
#lets see the population
population

In [ ]:
def draw_scatter_plot(texto=''):
    ##prepare scatter
    xx=list()
    xx[:]=()
    yy=list()
    yy[:]=()

    best_individual=0
    best_individual_fitness= calculate_fitness(population[0])

    tot=0
    
    for i in range(0,nindividuals):
        cfit=calculate_fitness(population[i])
        tot=tot+cfit
        if cfit<best_individual_fitness:
            best_individual=i
            best_individual_fitness=calculate_fitness(population[i])


    for i in population[best_individual]:
        xx.append(city_list[i][1])
        yy.append(city_list[i][2])

    print("Best individual is number " + str(best_individual) + " with a fitness of " + str(best_individual_fitness))
    print("Average fitness is " + str(tot/len(population)))

    plt.pyplot.scatter(xx,yy,c='r',marker='o')
    plt.pyplot.plot(xx,yy)
    
    #This is hardcoded to the cities example if you are using another file make sure the positions are still valid
    plt.pyplot.text(1,6000,texto + '  Best fitness :' + str(best_individual_fitness))
    plt.pyplot.show()

In [ ]:
#this is the best solution on the randomized initial population
draw_scatter_plot()

In [ ]:
bxplot=list()
bxplot[:]=[]

max_iterations=max_iterations

f = FloatProgress(min=0, max=max_iterations) # instantiate the bar
display(f) # display the bar
f.value=0

for j in range(0,max_iterations):
    #initialize empty population
    tmp_population=np.zeros((nindividuals,ncities),dtype=int)
    
    bxpltseries=list()
    bxpltseries[:]=[]
    
    for i in range(0,int(nindividuals/2)):
        tmpa = tournament_selection(population,tournamentsize)
        tmpa_fitness = calculate_fitness(tmpa)

        tmpb = tournament_selection(population,tournamentsize)
        tmpb_fitness = calculate_fitness(tmpb)
        
        if cx_probability>random():
            tmpc,tmpd=crossover(tmpa,tmpb)
        else:
            tmpc,tmpd=tmpa[:],tmpb[:]
        
        #this is used to create a boxplot in the end of the runs
        bxpltseries.append(tmpa_fitness)
        bxpltseries.append(tmpb_fitness)

        if mx_probability>random():
            tmp2=mutate(tmpc)
            tmp_population[i]=tmp2[:]
        
            tmp3=mutate(tmpd)
            tmp_population[int(nindividuals/2)+i]=tmp3[:]
        
        else:
            tmp_population[i]=tmpc[:]
            tmp_population[int(nindividuals/2)+i]=tmpd[:]
    
    if j%5==0:
        print('------' + str(j))
        bxplot.append(bxpltseries)
        draw_scatter_plot('Generation : ' + str(j))
    
    population=tmp_population[:]
    f.value += 1
    
f.close() #hide progressbar

### Visualize the resulting fitnesses

In [ ]:
plt.pyplot.boxplot(bxplot)

plt.pyplot.show()

The figure above shows a typical hockeystick plot.

In [ ]:
plt.rcParams["figure.figsize"]=(64,32)
plt.pyplot.boxplot(bxplot)

plt.pyplot.show()